In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential 
import cv2

import scipy.io as sio
import pandas as pd

# If you cant run this run the following commands in terminal
# python -m pip install -U pip
# python -m pip install scipy
# pip install opencv-python
# python3 -m pip install tensorflow
# python -m pip install -U matplotlib


In [5]:
car_annotations = sio.loadmat("cars_test_annos_with_labels.mat")

annos = car_annotations['annotations']
annos = np.transpose(annos)

file_names = []
bboxes= []

for annotation in annos:
    bbox_x1 = annotation[0][0][0][0]
    bbox_y1 = annotation[0][1][0][0]
    bbox_x2 = annotation[0][2][0][0]
    bbox_y2 = annotation[0][3][0][0]
    fname = annotation[0][5][0]
    car_class = annotation[0][4][0]
    bboxes.append((fname,bbox_x1, bbox_x2, bbox_y1, bbox_y2, int(list(car_class)[0])))

test_meta = pd.DataFrame(bboxes, columns= ['fnames', 'bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2', 'car_class'])


test_meta

,fnames,bbox_x1,bbox_x2,bbox_y1,bbox_y2,car_class
0,00001.jpg,30,246,52,147,181
1,00002.jpg,100,576,19,203,103
2,00003.jpg,51,968,105,659,145
3,00004.jpg,67,581,84,407,187
4,00005.jpg,140,593,151,339,185
...,...,...,...,...,...,...
8036,08037.jpg,49,1169,57,669,63
8037,08038.jpg,23,640,18,459,16
8038,08039.jpg,33,602,27,252,17
8039,08040.jpg,33,521,142,376,38


In [6]:
cars_annos = sio.loadmat('devkit/cars_train_annos.mat')
annotations = cars_annos['annotations']
annotations = np.transpose(annotations)

fnames = []
bboxes = []

for annotation in annotations:
    bbox_x1 = annotation[0][0][0][0]
    bbox_y1 = annotation[0][1][0][0]
    bbox_x2 = annotation[0][2][0][0]
    bbox_y2 = annotation[0][3][0][0]
    fname = annotation[0][5][0]
    car_class = annotation[0][4][0]
    bboxes.append((fname,bbox_x1, bbox_x2, bbox_y1, bbox_y2, int(list(car_class)[0])))
    
    
train_meta = pd.DataFrame(bboxes, columns = ['fnames','bbox_x1', 'bbox_x2', 'bbox_y1', 'bbox_y2','car_class'])

train_meta.head()

,fnames,bbox_x1,bbox_x2,bbox_y1,bbox_y2,car_class
0,00001.jpg,39,569,116,375,14
1,00002.jpg,36,868,116,587,3
2,00003.jpg,85,601,109,381,91
3,00004.jpg,621,1484,393,1096,134
4,00005.jpg,14,133,36,99,106


In [36]:
"""
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE

from PIL import Image

for i in range(0, len(train_meta)):
    img_path = "cars_train/cars_train/"+train_meta.fnames[i]
    
    left = train_meta.bbox_x1[i]
    top = train_meta.bbox_y1[i]
    right = train_meta.bbox_x2[i]
    bottom = train_meta.bbox_y2[i]
    
    img= Image.open(img_path)
    img = img.crop((left, top, right, bottom))
    img_output = "./training" + train_meta.fnames[i]
    img.save(img_output)


DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
"""

In [39]:
"""
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE


for i in range(0, len(test_meta)):
    img_path = "cars_test/cars_test/"+test_meta.fnames[i]
    
    left = test_meta.bbox_x1[i]
    top = test_meta.bbox_y1[i]
    right = test_meta.bbox_x2[i]
    bottom = test_meta.bbox_y2[i]
    
    img= Image.open(img_path)
    img = img.crop((left, top, right, bottom))
    img_output = "./testing" + train_meta.fnames[i]
    img.save(img_output)



DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
DO NOT RUN PLEASE
"""

In [7]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import glob
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization
import keras

In [8]:
train_meta = train_meta.astype({"car_class": str}, errors='raise')
test_meta = test_meta.astype({"car_class": str}, errors='raise') 

train_meta['fnames'] = "training" + train_meta["fnames"]
test_meta['fnames'] = "testing" + test_meta["fnames"]

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, validation_split = .20)
training_set = train_datagen.flow_from_dataframe(
dataframe= train_meta,
directory= "./cropped/training//",
x_col= 'fnames',
y_col='car_class',
subset = "training")

validation_set = train_datagen.flow_from_dataframe(
    dataframe = train_meta,
    directory ="./cropped/training//",
    x_col= 'fnames',
    y_col='car_class',
    subset = "validation")

test_datagen = ImageDataGenerator( rescale=1./255,horizontal_flip=True)

test_set = train_datagen.flow_from_dataframe(
dataframe= test_meta,
directory= "./cropped/testing//",
x_col= 'fnames',
y_col='car_class')

Found 6516 validated image filenames belonging to 196 classes.
Found 1628 validated image filenames belonging to 196 classes.
Found 8041 validated image filenames belonging to 196 classes.


In [11]:
model = Sequential()

model.add(Conv2D(255, (3, 3), input_shape=(255, 255, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(255, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(255, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(196,activation='softmax'))

# model.summary()

In [20]:
optimizer = keras.optimizers.Adam(learning_rate =.01)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

hist = model.fit(x = training_set , validation_data = validation_set , epochs = 10)


Epoch 1/10


KeyboardInterrupt: 

In [15]:
model.evaluate(test_set)

252/252 [==============================] - 1581s 6s/step - loss: 5.2754


5.275388717651367